In [5]:
from typing import List

import numpy as np

In [2]:
class Param:
    def __init__(self, name: str, distribution):
        self.name = name
        self.distribution = distribution

    def create_value(self):
        return self.distribution()

In [4]:
param = Param("alpha", lambda: np.random.normal(0, 1))
param.create_value()

0.2295171895586531

In [7]:
class Stage:
    def __init__(self, name: str, params: List):
        self.name = name
        self.params = params


class Pipline:
    def __init__(self, stages: List[Stage]):
        self.stages = stages

In [6]:
from abc import ABC, abstractmethod


class StageType(ABC):
    @property
    @abstractmethod
    def name(self) -> str:
        pass

    @property
    @abstractmethod
    def params(self) -> List[Param]:
        pass

    def get_stage(self) -> Stage:
        return Stage(self.name, [param.create_value() for param in self.params])

In [ ]:
size_param = Param("size", lambda: np.random.randint(1, 5))


class ColorStage(StageType):
    def __init__(self, color: str):
        self.color = color

    @property
    def name(self) -> str:
        return self.color

    @property
    def params(self) -> List[Param]:
        return [size_param]


red_stage = ColorStage("red")
green_stage = ColorStage("green")
blue_stage = ColorStage("blue")


def evaluate(pipeline: Pipline):
    res = 0
    i = 0
    n = len(pipeline.stages)
    while i < n:
        stage = pipeline.stages[i]
        if stage.name == "red" and i < n - 2 and \
                pipeline.stages[i + 1].name == "green" and \
                pipeline.stages[i + 2].name == "blue":
            res += stage.params[0]
                        
        